In [ ]:
!pip install transformers torch hf_transfer

In [3]:
import os
import sys
import subprocess
from huggingface_hub import login

# Add the cloned repository to the Python path
sys.path.append('repository/circuit-tracer')
sys.path.append('repository/circuit-tracer/demos')

# This will prompt you for your token
login()


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
model_name = 'google/gemma-2-2b'
model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Example 1 - Multi-hop Reasoning

In [9]:
input_text = "The capital of state containing Dallas is"
input_ids = tokenizer(input_text, return_tensors="pt")#.to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))

<bos>The capital of state containing Dallas is Austin.

The capital of state containing Dallas is Austin.

The capital of state containing Dallas is Austin.

The capital of state containing Dallas is Austin.


In [11]:
prompt = (
  "Let's think step by step and answer the following question. "
  "Question: The capital of the state containing Dallas is"
)
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
out = model.generate(input_ids, max_new_tokens=48, do_sample=False, temperature=0.0)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Let's think step by step and answer the following question. Question: The capital of the state containing Dallas is Austin. The capital of the state containing Houston is Austin. The capital of the state containing San Antonio is Austin. The capital of the state containing San Diego is Austin. The capital of the state containing San Francisco is Austin. The capital of


In [20]:
prompt = (
  "What is the capital of the state containing Dallas? Let me solve this step by step."
)
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
out = model.generate(input_ids, max_new_tokens=125, do_sample=False, temperature=0.0)
print(tokenizer.decode(out[0], skip_special_tokens=True))

What is the capital of the state containing Dallas? Let me solve this step by step.

Step 1:

The state containing Dallas is Texas.

Step 2:

The capital of Texas is Austin.

Step 3:

The capital of Texas is Austin.

Step 4:

The capital of Texas is Austin.

Step 5:

The capital of Texas is Austin.

Step 6:

The capital of Texas is Austin.

Step 7:

The capital of Texas is Austin.

Step 8:

The capital of Texas is Austin.

Step 9:

The capital of Texas is Austin.

Step 10:

The


### Example 2 - GSM8K

In [22]:
input_text = "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?" #Answer: $10
input_ids = tokenizer(input_text, return_tensors="pt")#.to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=10)
print(tokenizer.decode(outputs[0]))

<bos>Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

A. $60

B. $


In [21]:
input_text = "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?" #Answer: $10
input_ids = tokenizer(input_text, return_tensors="pt")#.to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=125)
print(tokenizer.decode(outputs[0]))

<bos>Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

A. $60

B. $62

C. $64

D. $66

Show more
Step 1
1 of 2

$12\times 50=600 $

The amount of money earned is the product of the number of hours and the hourly rate.

Result
2 of 2

A<eos>


In [14]:
input_text = "Let's think step by step and answer the following question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?" #Answer: $10
input_ids = tokenizer(input_text, return_tensors="pt")#.to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=125)
print(tokenizer.decode(outputs[0]))

<bos>Let's think step by step and answer the following question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

Step 1
1 of 2

Let $x$ be the number of hours Weng worked.

The amount of money she earned is the product of the number of hours she worked and the hourly rate.

$\begin{align*} \text{amount of money earned} &= \text{number of hours worked} \times \text{hourly rate} \\ &= x \times \$12 \\ &= \$12x \end{align*} $

The amount of money she earned is $\$12x$.

Result
2 of 2

$\$12x $<eos>


In [18]:
prompt = (
  "James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year? Let me solve this step by step." #Answer: 624
)
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
out = model.generate(input_ids, max_new_tokens=150, do_sample=False, temperature=0.0)
print(tokenizer.decode(out[0], skip_special_tokens=True))

James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year? Let me solve this step by step.

Step 1: Find the number of weeks in a year.

There are 52 weeks in a year.

Step 2: Find the number of letters James writes in a year.

James writes 2 letters a week.

So, he writes 2 x 52 = 104 letters in a year.

Step 3: Find the number of pages James writes in a year.

James writes 3 pages in a letter.

So, he writes 3 x 104 = 312 pages in a year.

Therefore, James writes 312 pages in a year.


#### Gemma IT model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)